In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from openai import OpenAI
from key import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
movie_latest_small=pd.read_csv('../../data/processed_movie_latest_small.csv')
movie_latest_small.head()

,userId,movieId,title,genres,rating,timestamp,tag
0,1,1,Toy Story (1995),"Adventure, Animation, Children, Comedy, Fantasy",4.0,964982703,NaN
1,1,3,Grumpier Old Men (1995),"Comedy, Romance",4.0,964981247,NaN
2,1,6,Heat (1995),"Action, Crime, Thriller",4.0,964982224,NaN
3,1,47,Seven (a.k.a. Se7en) (1995),"Mystery, Thriller",5.0,964983815,NaN
4,1,50,"Usual Suspects, The (1995)","Crime, Mystery, Thriller",5.0,964982931,NaN


In [13]:
movie_latest_small.title.nunique()

9719

In [3]:
# for each user, find his top-most rated movies
N_TOP_MOVIES = 10
def get_top_recent_movies(movies_df, user_id, num_movies=N_TOP_MOVIES):
    user_ratings = movies_df[movies_df['userId'] == user_id]
    user_ratings = user_ratings.sort_values(by=['rating', 'timestamp'], ascending=[False, False])
    return user_ratings.head(num_movies)

# get the top 5 movies for all users
top_movies = pd.DataFrame()
for user_id in movie_latest_small['userId'].unique():
    top_movies = pd.concat([top_movies, get_top_recent_movies(movie_latest_small, user_id, 10)])
top_movies.reset_index(drop=True, inplace=True)
top_movies.head(15)

,userId,movieId,title,genres,rating,timestamp,tag
0,1,553,Tombstone (1993),"Action, Drama, Western",5.0,964984153,NaN
1,1,157,Canadian Bacon (1995),"Comedy, War",5.0,964984100,NaN
2,1,1298,Pink Floyd: The Wall (1982),"Drama, Musical",5.0,964984086,NaN
3,1,3053,"Messenger: The Story of Joan of Arc, The (1999)","Drama, War",5.0,964984086,NaN
4,1,3448,"Good Morning, Vietnam (1987)","Comedy, Drama, War",5.0,964984054,NaN
5,1,151,Rob Roy (1995),"Action, Drama, Romance, War",5.0,964984041,NaN
6,1,1224,Henry V (1989),"Action, Drama, Romance, War",5.0,964984018,NaN
7,1,527,Schindler's List (1993),"Drama, War",5.0,964984002,NaN
8,1,5060,M*A*S*H (a.k.a. MASH) (1970),"Comedy, Drama, War",5.0,964984002,NaN
9,1,3147,"Green Mile, The (1999)","Crime, Drama",5.0,964983873,NaN


In [20]:
from imdb import IMDb


def get_movie_summary(title, max_length=150):
    ia = IMDb()
    # Search for the movie
    search_results = ia.search_movie(title)
    if not search_results:
        return "Summary not found"
    
    # Get the first movie from the search results
    movie = search_results[0]
    ia.update(movie, info=['main'])
    
    # Get the summary
    summary = movie.get('plot outline')
    if summary:
        # Truncate the summary to the max_length
        if len(summary) > max_length:
            truncated_summary = summary[:max_length] + '... [truncated]'
        else:
            truncated_summary = summary
        return truncated_summary
    return "Summary not found"


In [21]:
movie_wiki = pd.DataFrame()
for idx, title in enumerate(top_movies['title'].unique()):
    movie_wiki = pd.concat([movie_wiki, pd.DataFrame({'title': [title], 'summary': [get_movie_summary(title)]})])
    if idx % 10 == 0:
        print(f"Processed {idx} movies")
movie_wiki.head()


Processed 0 movies
Processed 10 movies
Processed 20 movies


KeyboardInterrupt: 

In [22]:
pd.set_option('display.max_colwidth', None)
movie_wiki

,title,summary
0,Tombstone (1993),"After success cleaning up Dodge City, Wyatt Earp moves to Tombstone, Arizona, and wishes to get rich in obscurity. He meets his brothers there, as wel... [truncated]"
0,Canadian Bacon (1995),"The US economy is in a rut, and so is the president's approval rating. What we need is a good war, but the Russians aren't interested. Hey -- how abou... [truncated]"
0,Pink Floyd: The Wall (1982),"Rock star Pink Floyd is a tortured soul. Because of his childhood, he has always tried to make meaningful emotional connections to other living creatu... [truncated]"
0,"Messenger: The Story of Joan of Arc, The (1999)","1429. While the war between France and England (the Hundred Years War) appeared settled in 1420, in England's favour, the death of King Henry V of Eng... [truncated]"
0,"Good Morning, Vietnam (1987)",A new Disc Jockey is shipped from Crete to Vietnam to bring humor to Armed Forces Radio. He turns the studio on its ear and becomes wildly popular wit... [truncated]
0,Rob Roy (1995),"In the highlands of Scotland in the 1700s, Rob Roy tries to lead his small town to a better future, by borrowing money from the local nobility to buy ... [truncated]"
0,Henry V (1989),"King Henry V of England (Sir Kenneth Branagh) is insulted by King Charles VI of France (Paul Scofield). As a result, he leads his army into battle aga... [truncated]"
0,Schindler's List (1993),Oskar Schindler is a vain and greedy German businessman who becomes an unlikely humanitarian amid the barbaric German Nazi reign when he feels compell... [truncated]
0,M*A*S*H (a.k.a. MASH) (1970),The personnel at the 4077 MASH unit deal with the horrors of the Korean War and the stresses faced in surgery by whatever means. The tone at the MASH ... [truncated]
0,"Green Mile, The (1999)",Based on Stephen King's 1996 novel of the same name. Stars Tom Hanks as a death row corrections officer during the U.S. Great Depression who witnesses... [truncated]


In [4]:
import re

def extract_movie_titles(response):
    # Extract lines that match the format [Title (Year)]
    pattern = r"\[.*\]"
    titles = re.findall(pattern, response)
    return [title.strip("[]") for title in titles]

In [5]:
""" Number of users to run the recommender on
    (Use a small sample for faster results)
    If you want to run the recommender on all users, uncomment the last line
"""
num_users = 5

# Uncomment below to run on all users
# num_users = len(prompts_1)

In [6]:
def openai_response(prompts, num_users, output_file='responses.json'):

    responses = []
    for i in range(num_users):
        response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= [
            { 
            'role':'system','content':'You are an AI assistant specialized in movie recommendations.'},
            {'role':'user','content' : prompts[i]
            }
        ],
        temperature=0,
        )
        responses.append(response)

    # parse the responses
    responses_recommendations = [extract_movie_titles(response.choices[0].message.content) for response in responses]

    # write the responses to a file
    with open(output_file, 'w') as f:
        for i in range(num_users):
            f.write(str(responses_recommendations[i]) + '\n')
        
        print(f"Response written to {output_file}")
    return responses_recommendations

# calculate the hit rate
# hit rate for the recommendations
def hit_rate(recommendation, actual):
    hit = 0
    for rec in recommendation:
        if rec in actual:
            hit += 1
    return hit/len(recommendation)

# NDCG for the recommendations
def ndcg(recommendation, actual):
    dcg = 0
    for i, rec in enumerate(recommendation):
        if rec in actual:
            dcg += 1/np.log2(i+2)
    idcg = 0
    for i in range(len(actual)):
        idcg += 1/np.log2(i+2)
    return dcg/idcg


def eval_response(responses_recommendations, selected_movie_data, original_movie_data, num_users):
    hit_rates = []
    ndcg_scores = []
    for i in range(num_users):
        user_id = selected_movie_data['userId'].unique()[i]
        user_movies = original_movie_data[original_movie_data['userId']==user_id]
        user_watched_movies = user_movies['title'].values
        # print(f"User {user_id} watched movies: {user_watched_movies}")
        recommendation = responses_recommendations[i]
        # print(f"Recommendation: {recommendation}")
        hit_rate_ = hit_rate(recommendation, user_watched_movies)
        # print(f"Hit Rate: {hit_rate_}")
        hit_rates.append(hit_rate_)
        ndcg_ = ndcg(recommendation, user_watched_movies)
        # print(f"NDCG: {ndcg_}")
        ndcg_scores.append(ndcg_)

    avg_hit_rate = np.mean(hit_rates)
    avg_ndcg = np.mean(ndcg_scores)

    print(f"Average Hit Rate: {avg_hit_rate}")
    print(f"Average NDCG: {avg_ndcg}")

    return hit_rates, ndcg_scores


In [7]:
# read the responses from the json file
def read_responses(output_file):
    responses_recommendations = []
    with open(output_file, 'r') as f:
        for line in f:
            responses_recommendations.append(eval(line))
    return responses_recommendations

In [13]:
def generate_prompt_from_df(df, user_id, num):
    df_user = df[df['userId'] == user_id]
    movies_list = '\n'.join([f"Movie {i+1}: {row['title']}, Genre: {row['genres']}, Rating: {row['rating']}" for i, row in df_user.iterrows()])
    
    prompt_template = '''
I am user {user_id}. My most recent {num} top-rated movies are listed below along with their genres and ratings:
{movies_list}
Please recommend 10 movies similar to these that I haven't watched yet. Format your response as a list of movie titles, using brackets around the titles and separating each title with a new line for easy parsing.
Example format:
Here are the 10 movies recommended for you: 
[Midnight Cowboy (1969)]
[Lost in Translation (2003)]
[etc.]

Answer:
    '''

    prompt = prompt_template.format(user_id=user_id, num=num, movies_list=movies_list)
    return prompt

In [14]:
prompts_genre = []
for user_id in top_movies['userId'].unique():
    prompt_genre = generate_prompt_from_df(top_movies, user_id, N_TOP_MOVIES)
    prompts_genre.append(prompt_genre)

prompts_genre[:2]

["\nI am user 1. My most recent 10 top-rated movies are listed below along with their genres and ratings:\nMovie 1: Tombstone (1993), Genre: Action, Drama, Western, Rating: 5.0\nMovie 2: Canadian Bacon (1995), Genre: Comedy, War, Rating: 5.0\nMovie 3: Pink Floyd: The Wall (1982), Genre: Drama, Musical, Rating: 5.0\nMovie 4: Messenger: The Story of Joan of Arc, The (1999), Genre: Drama, War, Rating: 5.0\nMovie 5: Good Morning, Vietnam (1987), Genre: Comedy, Drama, War, Rating: 5.0\nMovie 6: Rob Roy (1995), Genre: Action, Drama, Romance, War, Rating: 5.0\nMovie 7: Henry V (1989), Genre: Action, Drama, Romance, War, Rating: 5.0\nMovie 8: Schindler's List (1993), Genre: Drama, War, Rating: 5.0\nMovie 9: M*A*S*H (a.k.a. MASH) (1970), Genre: Comedy, Drama, War, Rating: 5.0\nMovie 10: Green Mile, The (1999), Genre: Crime, Drama, Rating: 5.0\nPlease recommend 10 movies similar to these that I haven't watched yet. Format your response as a list of movie titles, using brackets around the titles 

In [15]:
prompt_role_rec = openai_response(prompts_genre, num_users, output_file='responses_add_role.json')

hit_rates_role, ndcg_scores_role = eval_response(prompt_role_rec, top_movies, movie_latest_small, num_users)

Response written to responses_add_role.json
Average Hit Rate: 0.2
Average NDCG: 0.0633117762790104


## 2-step

In [23]:
from imdb import IMDb

def get_movie_summary(title):
    ia = IMDb()
    # Search for the movie
    search_results = ia.search_movie(title)
    if not search_results:
        return "Summary not found"
    
    # Get the first movie from the search results
    movie = search_results[0]
    ia.update(movie, info=['main'])
    
    # Get the summary
    summary = movie.get('plot outline')
    if summary:
        return summary
    return "Summary not found"

In [24]:
movies_summaries = pd.DataFrame()
for title in movie_titles:
    summary = get_movie_summary(title)
    movies_summaries = pd.concat([movies_summaries, pd.DataFrame({'title': [title], 'summary': [summary]})])
movies_summaries.reset_index(drop=True, inplace=True)
movies_summaries

AttributeError: 'DataFrame' object has no attribute 'append'